In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras import optimizers
import os.path

C:\Users\anton\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# This part simply illustrate the unsatisfactory behavior of the inception validation accuracy
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# dimensions of our images.
# Inception input size
img_width, img_height = 299, 299
train_data_dir = './train'
validation_data_dir = './valid'

# Misc.
nb_train_samples = 192171
nb_validation_samples = 6309
top_epochs = 200
batch_size = 128

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(128, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

top_layers_checkpoint_path = 'InceptionLogRegTop_cp.h5'

if os.path.exists(top_layers_checkpoint_path):
    model.load_weights(top_layers_checkpoint_path)
    print ("Checkpoint '" + top_layers_checkpoint_path + "' loaded.")

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

#Save the model after every epoch.
mc_top = ModelCheckpoint(top_layers_checkpoint_path,
                         monitor='val_acc', 
                         verbose=0,
                         save_best_only=True, 
                         save_weights_only=False, 
                         mode='auto', 
                         period=1)

# train the model on the new data for a few epochs
# model.fit_generator(...)
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples // batch_size,
    nb_epoch=top_epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples // batch_size,
    callbacks=[mc_top])

C:\Users\anton\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 192171 images belonging to 128 classes.
Found 6309 images belonging to 128 classes.


C:\Users\anton\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:73: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\anton\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:73: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., callbacks=[<keras.ca..., epochs=300, validation_steps=49, validation_data=<keras.pre..., steps_per_epoch=11)`


Epoch 1/300
11/11 [==============================] - 167s 15s/step - loss: 4.8305 - acc: 0.0213 - val_loss: 4.7827 - val_acc: 0.0552
Epoch 2/300
11/11 [==============================] - 152s 14s/step - loss: 4.3421 - acc: 0.1357 - val_loss: 4.2183 - val_acc: 0.1678
Epoch 3/300
11/11 [==============================] - 150s 14s/step - loss: 3.8685 - acc: 0.2344 - val_loss: 3.7754 - val_acc: 0.2310
Epoch 4/300
11/11 [==============================] - 151s 14s/step - loss: 3.4511 - acc: 0.3196 - val_loss: 3.4440 - val_acc: 0.2703
Epoch 5/300
11/11 [==============================] - 152s 14s/step - loss: 3.1589 - acc: 0.3594 - val_loss: 3.1447 - val_acc: 0.3189
Epoch 6/300
11/11 [==============================] - 152s 14s/step - loss: 2.8805 - acc: 0.4098 - val_loss: 2.8956 - val_acc: 0.3462
Epoch 7/300
11/11 [==============================] - 152s 14s/step - loss: 2.6414 - acc: 0.4311 - val_loss: 2.7786 - val_acc: 0.3527
Epoch 8/300
11/11 [==============================] - 151s 14s/step - 

KeyboardInterrupt: 